# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 192MB of 14542MB


# Import Customized modules and Create ImageJ Object

In [3]:
import os
import sys
import re
from typing import TextIO, Dict
from datetime import datetime
from glob import glob

import json
import logging
import numpy as np


sys.path.append("/home/rime97410000/ZebraFish_Code/ZebraFish_AP_POS/modules") # add path to scan customized module

from logger import init_logger
from fileop import create_new_dir
from lifcheck import check_image_name
from composite_ij import dump_info, save_tif_with_SN, group_show, group_hide

In [4]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] some ImageJ plugins need to be new before use
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator

rm = RoiManager()
imageCalculator = ImageCalculator()

In [5]:
log = init_logger(r"ImageJ_BF_analysis")

Test : Get ```seriesCount``` ( Method 1 )

In [6]:
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Ext = loci.plugins.macro.LociFunctions()

# LociFunctions = sj.jimport("loci.plugins.macro.LociFunctions")
# Ext = LociFunctions()

test_lif = r"C:\Users\confocal_microscope\Desktop\PyImageJ-test-data\BrightField_RAW\20220610_CE001_palmskin_8dpf.lif"

Ext.setId(test_lif)

seriesCount = jpype.java.lang.Double[1]
print(f'seriesCount {type(seriesCount)}: {seriesCount}')
Ext.getSeriesCount(seriesCount)
print(f'seriesCount {type(seriesCount)}: {seriesCount}')

seriesCount = seriesCount[0]
print(f'seriesCount {type(seriesCount)}: {seriesCount}')
Ext.close()

seriesCount <java class 'java.lang.Double[]'>: [None]
seriesCount <java class 'java.lang.Double[]'>: [6.0]
seriesCount <java class 'java.lang.Double'>: 6.0


Test : Get ```seriesCount``` ( Method 2 )

In [7]:
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# ImageReader = sj.jimport("loci.formats.ImageReader")
# Reader = ImageReader()

test_lif = r"C:\Users\confocal_microscope\Desktop\PyImageJ-test-data\BrightField_RAW\20220610_CE001_palmskin_8dpf.lif"

Reader.setId(test_lif)

seriesCount = Reader.getSeriesCount()
print(f'seriesCount {type(seriesCount)}: {seriesCount}')

seriesCount <java class 'JInt'>: 6


Test : RGB image name

In [8]:
# test_string = r"Series001_fish_11_palmskin_8dpf"
test_string = r"Series001_fish_207_F"


# Check format of 'image_name'
detect_old_new_list = re.findall("[BF]", test_string)
# print(detect_old_new_list)
image_name_list = re.split(" |_|-", test_string)
if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_bf", debug_mode=True)
else: failed_cnt, check_dict = check_image_name(image_name_list, "old_bf", debug_mode=True)


# Consociate image_name
if (check_dict["    format_and_type     "] == "old_bf") and (failed_cnt == 0): # name check passed, generate new name
    image_name_list.pop(3) # palmskin (old name only)
    image_name_list.pop(3) # [num]dpf (old name only)
    image_name_list.append("BF")
image_name = "_".join(image_name_list)


# log.info() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : log.info(check_dict["  len(image_name_list)  "])
if failed_cnt > 0 : log.info(check_dict["     failed warning     "])

# log_writer.write() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : print("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
if failed_cnt > 0: print("| {} \n".format(check_dict["     failed warning     "].strip()))

| 2023-04-22 00:16:29,840 | ImageJ_BF_analysis | INFO |       #### ERROR : image_name, At least 1 test failed: spelling of 'BF' 


failed_cnt = 1
check_dict \
{
  "    format_and_type     ": "new_bf",
  "  len(image_name_list)  ": "PASS",
  "  spelling of 'Series'  ": "PASS",
  "      Series[num]       ": "PASS",
  "   spelling of 'fish'   ": "PASS",
  "       fish_[ID]        ": "PASS",
  "    spelling of 'BF'    ": "FAILED --> 'F', misspelling of 'BF' ",
  "     failed warning     ": "      #### ERROR : image_name, At least 1 test failed: spelling of 'BF' "
}
| #### ERROR : image_name, At least 1 test failed: spelling of 'BF' 



# Start Process

Original file is ```20220614 macro for SL and SA measurement_by KY.ijm```

Modify by following :
- using pyimagej
- bug fix
- add **```WARNING```** and **```ERROR```** remind
- make log file pretty
- see more in [report](../../report/report.md#20230126)

In [9]:
def single_BF_Analysis(lif_path:str, total_lif_file:int, out_dir_root:str, 
                       log:logging.Logger, log_writer:TextIO, crop_rect:Dict[str, int],
                       micron_per_pixel:float, auto_threshold:str, measure_range:Dict[str, int]):
    
    
    log.info(f'LIF_FILE : {lif_path}')
    
    Reader.setId(lif_path)
    seriesCount = Reader.getSeriesCount()
    # log.info(f'seriesCount : {seriesCount}')
   

    for idx in range(seriesCount):
        
        SeriesNum = idx+1
        
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        
        # Consociate "BrightField_RAW" file_name
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        file_name_list = re.split(" |_|-", file_name)
        assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
        file_name = "_".join(file_name_list)
        
        # Get Prop("Image name")
        image_name = img.getProp("Image name")
        image_name = str(image_name)
        
        # Check format of 'image_name'
        detect_old_new_list = re.findall("[BF]", image_name)
        # print(detect_old_new_list)
        image_name_list = re.split(" |_|-", image_name)
        if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_bf")
        else: failed_cnt, check_dict = check_image_name(image_name_list, "old_bf")
        
        # Consociate image_name
        if (check_dict["    format_and_type     "] == "old_bf") and (failed_cnt == 0): # name check passed, generate new name
            image_name_list.pop(3) # palmskin (old name only)
            image_name_list.pop(3) # [num]dpf (old name only)
            image_name_list.append("BF")
        image_name = "_".join(image_name_list)
        
        # Combine 2 names above
        seN = f"{file_name} - {image_name}"
        
        # Get dimension info
        img_dimensions = img.getDimensions()
        # dim_1
        dim1_length = img.getNumericProperty("Image #0|DimensionDescription #1|Length")
        dim1_elem = img.getNumericProperty("Image #0|DimensionDescription #1|NumberOfElements")
        dim1_unit = dim1_length/(dim1_elem-1)*(10**6)
        # dim_2
        dim2_length = img.getNumericProperty("Image #0|DimensionDescription #2|Length")
        dim2_elem = img.getNumericProperty("Image #0|DimensionDescription #2|NumberOfElements")
        dim2_unit = dim2_length/(dim2_elem-1)*(10**6)
        assert dim1_unit == dim2_unit, f"Voxel_X != Voxel_Y, Voxel_X, Voxel_Y = ({dim1_unit}, {dim2_unit}) micron/pixel"
        log.info(f"series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames ), Voxel_X_Y : {dim1_unit:.4f} micron")
        
        # Print ERRORs when checking 'image_name'
        if failed_cnt == -1 : log.info(check_dict["  len(image_name_list)  "])
        if failed_cnt > 0 : log.info(check_dict["     failed warning     "])
        
        # Write Log
        log_writer.write(f"|-- processing ...  series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} in {SeriesNum}/{total_lif_file} \n")
        log_writer.write(f"|         {seN} \n")
        log_writer.write(f"|         Dimensions : {img_dimensions} ( width, height, channels, slices, frames ), Voxel_X_Y : {dim1_unit:.4f} micron \n")
        if failed_cnt == -1 : log_writer.write("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
        if failed_cnt > 0: log_writer.write("| {} \n".format(check_dict["     failed warning     "].strip()))
        
        
        # Start image preprocessing
        
        
        # processing var
        tif_save_cnt = 1
        tif_save_SNdigits = "02"
        
        
        # Create sub folder
        if "delete" in seN: subfolder = os.path.join(out_dir_root, "!~delete", seN)
        else: subfolder = os.path.join(out_dir_root, seN)
        dir_metaimg = os.path.join(subfolder, "MetaImage")
        create_new_dir(subfolder, display_in_CLI=False)
        create_new_dir(dir_metaimg, display_in_CLI=False)


        # Find focused slices

        #   Pick up focused slice if slices > 1
        #   Plugin ref : https://sites.google.com/site/qingzongtseng/find-focus
        #   Algorithm  : autofocus algorithm "Normalized variance"  (Groen et al., 1985; Yeo et al., 1993).
        slices = img_dimensions[3]
        if slices > 1:
            log.info("      #### WARNING : Number of Slices > 1, run ' Find focused slices ' ") # WARNING:
            # Write Log
            log_writer.write("| #### WARNING : Number of Slices > 1, run ' Find focused slices ' \n") # WARNING:
            ij.IJ.run(img, "Find focused slices", "select=100 variance=0.000 select_only")
            img = ij.WindowManager.getCurrentImage()
            # img.show()
            img.hide()
        # dump_info(img)
        tif_save_cnt = save_tif_with_SN(img, f"original_16bit", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)


        # Convert to 8-bit and Set Scale

        ij.IJ.run("Conversions...", "scale weighted")
        ij.IJ.run(img, "8-bit", "")


        # Consociate the unit ( relationship between micron and pixel )

        #   To prevent some series of images are in different scales, for example, "20220708_CE009_palmskin_8dpf.lif"
        #   Microscope Metadata : 1 pixel = 0.0000065 m = 6.5 micron
        ij.IJ.run(img, "Set Scale...", f"distance=1 known={micron_per_pixel} unit=micron")


        # Cropping image

        img.setRoi(crop_rect["x"], crop_rect["y"], crop_rect["w"], crop_rect["h"])
        crop_img = img.crop()
        tif_save_cnt = save_tif_with_SN(crop_img, f"cropped_BF", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        

        # Threshold
        
        crop_img_threshold = crop_img.duplicate()
        ij.IJ.run(crop_img_threshold, "Auto Threshold", f"method={auto_threshold} white")
        ij.prefs.blackBackground = True
        ij.IJ.run(crop_img_threshold, "Convert to Mask", "")
        tif_save_cnt = save_tif_with_SN(crop_img_threshold, f"AutoThreshold_{auto_threshold}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        

        # SL and SA measurement
        
        ij.IJ.run("Set Measurements...", "area mean min feret's display redirect=None decimal=2")
        ij.IJ.run(crop_img_threshold, "Analyze Particles...", "size={}-{} show=Masks display include add".format(measure_range["lower_bound"], measure_range["upper_bound"]))
        mask_img = ij.WindowManager.getCurrentImage()
        ij.IJ.run(mask_img, "Convert to Mask", "")
        tif_save_cnt = save_tif_with_SN(mask_img, f"measured_mask", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        mask_img.hide()
        
        rm.runCommand("Show All with labels")
        rm_size = int(rm.getCount())
        if rm_size > 0:
            
            if rm_size == 1:  # success to get fish
                mix_img = imageCalculator.run(crop_img, mask_img, "Average create")
                tif_save_cnt = save_tif_with_SN(mix_img, f"cropped_BF--MIX", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
                
                rm.save(os.path.normpath(f"{dir_metaimg}/RoiSet.zip"))
                ij.IJ.saveAs("Results", os.path.normpath(f"{subfolder}/AutoAnalysis.csv"))
            
            else:
                log.info(f'      ROI in RoiManager: {rm_size}')
                log.info(f'      #### ERROR : Number of ROI not = 1')
                # Write Log
                log_writer.write(f"|         number of ROI = {rm_size} \n")
                log_writer.write("| #### ERROR : Number of ROI not = 1 \n") # ERROR:

            rm.runCommand("Deselect")
            rm.runCommand("Delete") # delete ROI, otherwise, it increases infinitely
        
        ij.IJ.run("Clear Results", "")
        ij.IJ.run("Close All", "")
        log_writer.write(f"| \n") # make Log file looks better.

In [10]:
if int(rm.getCount()) > 0:
    rm.runCommand("Deselect")
    rm.runCommand("Delete")
    ij.IJ.run("Clear Results", "")
ij.IJ.run("Close All", "")


analysis_kwargs = {
    "crop_rect" : {"x": 50, "y": 700, "w":1950, "h":700}, # x, y is `Left-Top` pt
    "micron_per_pixel" : 3.25,
    "auto_threshold" : "Triangle",
    "measure_range" : {"lower_bound": 800000, "upper_bound": 4000000} # apply `Set Scale`
}

# File Input
bf_raw_lif_source = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230301_Update}_Zebrafish_A_P_strategies\BrightField_RAW\Upload_20230301"

# Result Output
ap_data_root = r"C:\Users\confocal_microscope\Desktop\{Temp}_Data\{20230305_NEW_STRUCT}_Academia_Sinica_i409"
analysis_method_desc = "KY_with_NameChecker"
analysis_root = os.path.join(ap_data_root, f"{{{analysis_method_desc}}}_BF_Analysis")
create_new_dir(analysis_root, display_in_CLI=False)


# Scan Leica LIF file
# lif_path_list = glob(os.path.normpath(f"{bf_raw_lif_source}/*/*.lif"))
lif_path_list = glob(os.path.normpath(f"{bf_raw_lif_source}/*.lif"))
log.info(f'lif_path_list [ found {len(lif_path_list)} files ]: {lif_path_list},  \n')


# Create log writer
time_stamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')
log_path = os.path.join(analysis_root, f"{{Logs}}_{{imagej_BF_Analysis}}_{time_stamp}.log")
log_writer = open(log_path, mode="w")
log.info(f'log_writer {type(log_writer)}: {log_writer} \n')


# Write `analysis_kwargs` to log
log_writer.write(f"analysis_kwargs = {json.dumps(analysis_kwargs, indent=4)} \n")


for i, lif_path in enumerate(lif_path_list):
    
    log.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    
    # Write Log
    log_writer.write("\n\n\n")
    log_writer.write(f"|-----------------------------------------  Processing ... {i+1}/{len(lif_path_list)}  ----------------------------------------- \n")
    log_writer.write(f"| \n")
    log_writer.write(f"|         LIF_FILE : {lif_path.split(os.sep)[-1]} \n")
    log_writer.write(f"| \n")

    # analyze current LIF_FILE
    single_BF_Analysis(lif_path, len(lif_path_list), analysis_root, log, log_writer, **analysis_kwargs)
    log.info("\n") # make CLI output looks better.


log.info(" -- finished --")
log_writer.write("\n\n\n-----------------------------------------  finished  ----------------------------------------- \n")
log_writer.close()
# rm.close()
# ij.WindowManager.closeAllWindows()

| 2023-04-22 00:16:29,946 | ImageJ_BF_analysis | INFO | lif_path_list [ found 3 files ]: ['C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230301_Update}_Zebrafish_A_P_strategies\\BrightField_RAW\\Upload_20230301\\20221205_AI006_palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230301_Update}_Zebrafish_A_P_strategies\\BrightField_RAW\\Upload_20230301\\20230209_AI008_palmskin_10dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230301_Update}_Zebrafish_A_P_strategies\\BrightField_RAW\\Upload_20230301\\20230213_AI009_palmskin_10dpf.lif'],  

| 2023-04-22 00:16:29,947 | ImageJ_BF_analysis | INFO | log_writer <class '_io.TextIOWrapper'>: <_io.TextIOWrapper name='C:\\Users\\confocal_microscope\\Desktop\\{Temp}_Data\\{20230305_NEW_STRUCT}_Academia_Sinica_i409\\{KY_with_NameChecker}_BF_Analysis\\{Logs}_{imagej_BF_Analysis}_20230422_00_16_29.log' mode='w' encoding='UTF-8'> 

| 2023-04-2

In [11]:
log_writer.close()

# Other Useful Methods 

```
ij.WindowManager.closeAllWindows()

ij.WindowManager.getCurrentImage()

img.getProperties()

ij.WindowManager.getActiveWindow()
```